In [1]:
from amuse.units import units
from amuse.lab import Huayno, nbody_system, new_galactics_model
from amuse.lab import Gadget2
from amuse.community.ph4.interface import ph4
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from amuse.couple import bridge
from amuse.community.hermite.interface import Hermite
from amuse.community.bhtree.interface import BHTree
from astropy.io import fits

In [2]:
%%time

# Milky Way := Mdisk = 4.5 Mbulge | Mhalo = 100 Mbulge
# Andromeda := Mdisk = 4-7 Mbulge | Mhalo = 87 Mbulge
# Mdisk = 5 Mbulge, Mhalo = 95 Mbulge

n_halo  = 20000
n_bulge = 10000
n_disk  = 10000
M_galaxy = 1e12 | units.MSun
R_galaxy = 80  | units.kpc
converter = nbody_system.nbody_to_si(M_galaxy, R_galaxy)

galaxy1 = new_galactics_model(n_halo,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)
galaxy1.move_to_center()
# Order of particles: Disk => Bulge => Halo

CPU times: user 13.2 s, sys: 2.37 s, total: 15.6 s
Wall time: 17.8 s


In [3]:
%%time
M_galaxy_2 = 1e12 | units.MSun
R_galaxy_2 = 80  | units.kpc
converter_2 = nbody_system.nbody_to_si(M_galaxy_2, R_galaxy_2)
galaxy2 = new_galactics_model(n_halo,
                                converter_2,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)
galaxy2.move_to_center()

CPU times: user 12.7 s, sys: 2.27 s, total: 15 s
Wall time: 15 s


In [4]:
%%time
galaxy1.rotate(0., np.pi/2, 0.)
galaxy2.x  += 400 | units.kpc
galaxy2.vx += -500 |units.kms

CPU times: user 13.6 ms, sys: 2.64 ms, total: 16.2 ms
Wall time: 8.14 ms


In [5]:
converter = nbody_system.nbody_to_si(1e12|units.MSun, 100|units.kpc)
dynamics = BHTree(converter) # ph4 Does the trick, but is kinda slow
dynamics.parameters.epsilon_squared = (100|units.parsec)**2
dynamics.parameters.timestep = 1 |units.Myr
set1 = dynamics.particles.add_particles(galaxy1)
set2 = dynamics.particles.add_particles(galaxy2)

In [6]:
%%time
n_halo_test  = 20000
n_disk_test  = 10000
n_bulge_test = 10000

test_particles_1 = new_galactics_model(n_halo_test,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge_test,
                                disk_number_of_particles=n_disk_test)

test_particles_2 = new_galactics_model(n_halo_test,
                                converter_2,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge_test,
                                disk_number_of_particles=n_disk_test)
test_particles_1.move_to_center()
test_particles_2.move_to_center()

CPU times: user 26.1 s, sys: 4.75 s, total: 30.9 s
Wall time: 31.7 s


In [7]:
#set the mass of the test particles to 0
test_particles_1.mass = 0 |units.kg
test_particles_2.mass = 0 |units.kg

#we're only interested in the particles from the stars, and not the halo.
test_particles_1_stars = test_particles_1[:int(n_bulge_test+n_disk_test)]
test_particles_2_stars = test_particles_2[:int(n_bulge_test+n_disk_test)]

#rotate and translate in the same way as the galaxies
test_particles_1_stars.rotate(0., np.pi/2, 0.)
test_particles_2_stars.x  += 400 | units.kpc
test_particles_2_stars.vx += -500 |units.kms

star_dynamics = BHTree(converter)
star_set_1 = star_dynamics.particles.add_particles(test_particles_1_stars)
star_set_2 = star_dynamics.particles.add_particles(test_particles_2_stars)
star_dynamics.parameters.timestep = 1|units.Myr

In [8]:
gravity = bridge.Bridge(use_threading=False)
#gravity.add_system(star_dynamics, (dynamics,) )
gravity.add_system(dynamics)
channel = star_dynamics.particles.new_channel_to(star_dynamics.particles)

In [ ]:
gravity.timestep = 1|units.Myr
threshold = 20. |units.Myr 

times = np.arange(0., 7000, 1) | units.Myr
for time in tqdm(range(len(times))):
    gravity.evolve_model(times[time])
    if times[time] %threshold == 0|units.Myr:
        channel.copy()
        #x1 = set1.x[:20000].value_in(units.kpc)
        #y1 = set1.y[:20000].value_in(units.kpc)
        #z1 = set1.z[:20000].value_in(units.kpc)
        #x2 = set2.x[:20000].value_in(units.kpc)
        #y2 = set2.y[:20000].value_in(units.kpc)
        #z2 = set2.z[:20000].value_in(units.kpc)



        #fig = plt.figure(figsize = (10,8))
        #ax = Axes3D(fig)
        #ax.scatter(x1,y1,z1, s = 0.5)
        #ax.scatter(x2,y2,z2, s = 0.5)

        #ax.set_xlabel("x")
        #ax.set_ylabel("y")
        #ax.set_zlabel("z")
        #plt.show()
        
        #saving the orbit of the stars with mass
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_bulge+n_disk)].value_in(units.kpc), set1.y[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Big galaxy')
        plt.scatter(set2.x[:int(n_bulge+n_disk)].value_in(units.kpc), set2.y[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Small galaxy')
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig('./plots_2/merge_plots/snapshots_xy/snap%04d.png'%time)
        #plt.show()
        plt.close()
                
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_bulge+n_disk)].value_in(units.kpc), set1.z[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Big galaxy')
        plt.scatter(set2.x[:int(n_bulge+n_disk)].value_in(units.kpc), set2.z[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Small galaxy')
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig('./plots_2/merge_plots/snapshots_xz/snap%04d.png'%time)
        #plt.show()
        plt.close()
        
        #saving the orbit of the massless tracers
        plt.figure(figsize = (10,8))
        plt.scatter(star_set_1.x[:int(n_disk_test)].value_in(units.kpc), star_set_1.y[:int(n_disk_test)].value_in(units.kpc), s=1,label = 'Big galaxy (disk)',color = 'b')
        plt.scatter(star_set_2.x[:int(n_disk_test)].value_in(units.kpc), star_set_2.y[:int(n_disk_test)].value_in(units.kpc), s=1,label = 'Small galaxy (disk)', color ='k')
        plt.scatter(star_set_1.x[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), star_set_1.y[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), s=1,label = 'Big galaxy (bulge)',color = 'r')
        plt.scatter(star_set_2.x[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), star_set_2.y[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), s=1,label = 'Small galaxy bulge',color = 'magenta')
        plt.title("Disk + Bulge for test particles \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig('./plots_2/star_plots/snapshots_xy/snap%04d.png'%time)
        plt.close()
                
            
        plt.figure(figsize = (10,8))
        plt.scatter(star_set_1.x[:int(n_disk_test)].value_in(units.kpc), star_set_1.z[:int(n_disk_test)].value_in(units.kpc), s=1,label = 'Big galaxy (disk)',color = 'b')
        plt.scatter(star_set_2.x[:int(n_disk_test)].value_in(units.kpc), star_set_2.z[:int(n_disk_test)].value_in(units.kpc), s=1,label = 'Small galaxy (disk)', color ='k')
        plt.scatter(star_set_1.x[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), star_set_1.z[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), s=1,label = 'Big galaxy (bulge)',color = 'r')
        plt.scatter(star_set_2.x[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), star_set_2.z[int(n_disk_test):int(n_bulge_test+n_disk_test)].value_in(units.kpc), s=1,label = 'Small galaxy bulge',color = 'magenta')
        plt.title("Disk + Bulge for test particles \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig('./plots_2/star_plots/snapshots_xz/snap%04d.png'%time)
        plt.close()
        
        file = open("time.txt","w")
        file.write(str(times[time]))
        file.write("\n")
        #put the last data in fits files
        col1 = fits.Column(name='star_1_x', format='E', array=np.array(star_set_1.x.value_in(units.m)))
        col2 = fits.Column(name='star_1_y', format='E', array=np.array(star_set_1.y.value_in(units.m)))
        col3 = fits.Column(name='star_1_z', format='E', array=np.array(star_set_1.z.value_in(units.m)))
        col4 = fits.Column(name='star_1_vx', format='E', array=np.array(star_set_1.vx.value_in(units.kms)))
        col5 = fits.Column(name='star_1_vy', format='E', array=np.array(star_set_1.vy.value_in(units.kms)))
        col6 = fits.Column(name='star_1_vz', format='E', array=np.array(star_set_1.vz.value_in(units.kms)))

        col11 = fits.Column(name='star_2_x', format='E', array=np.array(star_set_2.x.value_in(units.m)))
        col12 = fits.Column(name='star_2_y', format='E', array=np.array(star_set_2.y.value_in(units.m)))
        col13 = fits.Column(name='star_2_z', format='E', array=np.array(star_set_2.z.value_in(units.m)))
        col14 = fits.Column(name='star_2_vx', format='E', array=np.array(star_set_2.vx.value_in(units.kms)))
        col15 = fits.Column(name='star_2_vy', format='E', array=np.array(star_set_2.vy.value_in(units.kms)))
        col16 = fits.Column(name='star_2_vz', format='E', array=np.array(star_set_2.vz.value_in(units.kms)))
        
        col21 = fits.Column(name='set_1_x', format='E', array=np.array(set1.x.value_in(units.m)))
        col22 = fits.Column(name='set_1_y', format='E', array=np.array(set1.y.value_in(units.m)))
        col23 = fits.Column(name='set_1_z', format='E', array=np.array(set1.z.value_in(units.m)))
        col24 = fits.Column(name='set_1_vx', format='E', array=np.array(set1.vx.value_in(units.kms)))
        col25 = fits.Column(name='set_1_vy', format='E', array=np.array(set1.vy.value_in(units.kms)))
        col26 = fits.Column(name='set_1_vz', format='E', array=np.array(set1.vz.value_in(units.kms)))

        
        col31 = fits.Column(name='set_2_x', format='E', array=np.array(set2.x.value_in(units.m)))
        col32 = fits.Column(name='set_2_y', format='E', array=np.array(set2.y.value_in(units.m)))
        col33 = fits.Column(name='set_2_z', format='E', array=np.array(set2.z.value_in(units.m)))
        col34 = fits.Column(name='set_2_vx', format='E', array=np.array(set2.vx.value_in(units.kms)))
        col35 = fits.Column(name='set_2_vy', format='E', array=np.array(set2.vy.value_in(units.kms)))
        col36 = fits.Column(name='set_2_vz', format='E', array=np.array(set2.vz.value_in(units.kms)))

        cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col11,col12,col13,col14,col15,col16,col21,col22,col23,col24,col25,col26,col31,col32,col33,col34,col35,col36])
        hdu = fits.BinTableHDU.from_columns(cols)
        hdu.writeto('data.fits',overwrite =True)
        file.write(str(times[time]))
        file.close()
        
        
gravity.stop()

 96%|█████████▌| 6705/7000 [40:05:20<3:21:17, 40.94s/it] 

In [ ]:
file = fits.open("data.fits")
a = file[1].data

In [ ]:
x = a.field("star_1_x")
print(x[:n_disk_test+n_bulge_test])
#test_particles_1.x = x|units.m

In [ ]:
b = open("time.txt")
c = b.read()
print(c)

In [ ]:
a.field("set_1_x")

In [ ]:
galaxies_x = np.array(np.append(a.field("set_1_x"),a.field("set_2_x")),dtype = "float64")#/3.086e19
galaxies_y = np.array(np.append(a.field("set_1_y"),a.field("set_2_y")),dtype = "float64")#/3.086e19
galaxies_z = np.array(np.append(a.field("set_1_z"),a.field("set_2_z")),dtype = "float64")#/3.086e19
print(np.mean(galaxies_x)/3.086e19)
print(np.mean(galaxies_y)/3.086e19)
print(np.mean(galaxies_z)/3.086e19)

In [ ]:
galaxies_vx = np.array(np.append(a.field("set_1_vx"),a.field("set_2_vx")),dtype = "float64")
galaxies_vy = np.array(np.append(a.field("set_1_vy"),a.field("set_2_vy")),dtype = "float64")
galaxies_vz = np.array(np.append(a.field("set_1_vz"),a.field("set_2_vz")),dtype = "float64")
print(np.mean(galaxies_vx))
print(np.mean(galaxies_vy))
print(np.mean(galaxies_vz))

In [ ]:
stars_x = np.array(np.append(a.field("star_1_x")[:n_bulge+n_disk],a.field("star_2_x")[:n_bulge+n_disk]),dtype="float64")#/3.086e19
stars_y = np.array(np.append(a.field("star_1_y")[:n_bulge+n_disk],a.field("star_2_y")[:n_bulge+n_disk]),dtype="float64")#/3.086e19
stars_z = np.array(np.append(a.field("star_1_z")[:n_bulge+n_disk],a.field("star_2_z")[:n_bulge+n_disk]),dtype="float64")#/3.086e19
stars_vx = np.array(np.append(a.field("star_1_vx")[:n_bulge+n_disk],a.field("star_2_vx")[:n_bulge+n_disk]),dtype = "float64")
stars_vy = np.array(np.append(a.field("star_1_vy")[:n_bulge+n_disk],a.field("star_2_vy")[:n_bulge+n_disk]),dtype = "float64")
stars_vz = np.array(np.append(a.field("star_1_vz")[:n_bulge+n_disk],a.field("star_2_vz")[:n_bulge+n_disk]),dtype = "float64")

star_x_wrt_mean_galaxies = stars_x - np.mean(galaxies_x)
star_y_wrt_mean_galaxies = stars_y - np.mean(galaxies_y)
star_z_wrt_mean_galaxies = stars_z - np.mean(galaxies_z)
star_vx_wrt_mean_galaxies = stars_vx - np.mean(galaxies_vx)
star_vy_wrt_mean_galaxies = stars_vy - np.mean(galaxies_vy)
star_vz_wrt_mean_galaxies = stars_vz - np.mean(galaxies_vz)

In [ ]:
stars_x/3.086e19

In [ ]:
dist_stars = np.sqrt(star_x_wrt_mean_galaxies**2 + star_y_wrt_mean_galaxies**2 + star_z_wrt_mean_galaxies**2)
vel_stars  = np.sqrt(star_vx_wrt_mean_galaxies**2 + star_vy_wrt_mean_galaxies**2 + star_vz_wrt_mean_galaxies**2)

print(np.where(dist_stars > 300*3.086e19)[0])

G = 6.67e-11
total_mass_galaxies_in_kg = galaxy1.mass.sum().value_in(units.kg) + 0.5*galaxy2.mass.sum().value_in(units.kg)
def v_escape(r,M):
    return(np.sqrt(2*G*M/r))

In [ ]:
np.array(np.append(a.field("star_1_x")[:n_bulge+n_disk],a.field("star_2_x")[:n_bulge+n_disk]),dtype="float64")

In [ ]:
v_esc_stars = v_escape(dist_stars,total_mass_galaxies_in_kg) /1000
print(v_esc_stars[:50])
print(vel_stars[:50])
print()
print(np.where(vel_stars > v_esc_stars)[0])

In [ ]:
print(vel_stars[32374],v_esc_stars[32374])
print(vel_stars[34234],v_esc_stars[34234])
print(vel_stars[37014],v_esc_stars[37014])

In [ ]:
print(dist_stars[32374]/3.086e19)
print(dist_stars[34234]/3.086e19)
print(dist_stars[37014]/3.086e19)

In [ ]:
def v_crit(M1,M2,a):
    G  = 4.3009125e-6 # in units kpc/Msun (Km/s)^2
    v1 = M2 * np.sqrt(G/(a*(M1+M2)))
    q  = M2/M1 
    v2 = v1/q
    v_crit = abs(v1-v2) # in km/s
    return(v_crit)


def v_and_r_rel_stars(v1,v2,r1,r2):
    v_rel = np.sqrt((v1[0]-v2[0])**2+(v1[1]-v2[1])**2+(v1[2]-v2[2])**2)
    r_rel = np.sqrt((r1[0]-r2[0])**2+(r1[1]-r2[1])**2+(r1[2]-r2[2])**2)
    return(r_rel,v_rel)


def binding_energy(M1,M2,v1,v2,r1,r2):
    G   = 4.3009125e-6 # in units kpc/Msun (Km/s)^2
    r = np.sqrt(((r1-r2)**2).sum())
    v = np.sqrt(((v1-v2)**2).sum())
    E_b = 0.5*v**2*(M1*M2/(M1+M2)) - (G*M1*M2/r)
    return(E_b)

star_1_pos = np.array([stars_x[34234],stars_y[34234],stars_z[34234]])/3.086e19
star_2_pos = np.array([stars_x[37014],stars_y[37014],stars_z[37014]])/3.086e19
star_3_pos = np.array([stars_x[32374],stars_y[32374],stars_z[32374]])/3.086e19

star_1_vel = np.array([stars_vx[34234],stars_vy[34234],stars_vz[34234]])
star_2_vel = np.array([stars_vx[37014],stars_vy[37014],stars_vz[37014]])
star_3_vel = np.array([stars_vx[32374],stars_vy[32374],stars_vz[32374]])

v_rel,r_rel = v_and_r_rel_stars(star_1_vel,star_2_vel,star_1_pos,star_2_pos)

v_crit(10,20,1e-3)

#v_crit(10,20,r_rel)


In [ ]:
def binaries(m1,m2,v1,v2,r1,r2,hardness=10):
    G  = 4.3009125e-6 # in units kpc/Msun (Km/s)^2
    total_v1_squared = (v1**2).sum()
    total_v2_squared = (v2**2).sum()
    E_kin_total = 0.5*(m1*total_v1_squared + m2*total_v2_squared)
    print(E_kin_total)
    limit_energy = E_kin_total/(m1+m2) * hardness
    
    r = np.sqrt(((r1-r2)**2).sum())
    v_squared = ((v1-v2)**2).sum()
    E_b = ((G*(m1+m2))/r) - (0.5*v_squared)
    #E_b = 0.5*v_squared*(m1*m2/(m1+m2)) - (G*m1*m2/r)
    
    #print(v_squared,m1+m2)
    print(m1+m2,r,v_squared)
    print(E_b,limit_energy)
    if E_b > limit_energy:
        return(True)
    else:
        return(False)



In [ ]:
#checks whether 2 stars move in the same direction with roughly equal velocity
def moving_same_direction(v1,v2,eps_v =0.7):
    v_ratio = v1/v2
    print(v_ratio)
    factors_true = 0
    if v_ratio[0] > eps_v and v_ratio[0] < 1/eps_v:
        print("vx within ratio")
        factors_true += 1
    if v_ratio[1] > eps_v and v_ratio[1] < 1/eps_v:
        print("vy within ratio")
        factors_true += 1
    if v_ratio[2] > eps_v and v_ratio[2] < 1/eps_v:
        print("vz within ratio")
        factors_true += 1
    if factors_true >1:
        return(True)
    else:
        return(False)

def getting_closer(v1,v2,r1,r2,time = np.linspace(0,10000,1000),eps_r = 1.1):
    #convert v to kpc/Myr
    v_1 = v1*3.154e13/3.086e16
    v_2 = v2*3.154e13/3.086e16
    v_rel = v2-v1
    r_rel = r2-r1
    
    getting_closer = False
    moves_away_slightly = False
    for i in range(1,len(time)):
        r_rel_new = r_rel + v_rel*(time[i]-time[i-1])
        if (r_rel/eps_r).sum() < (v1 * (time[i]-time[i-1])).sum():
            moves_away_slightly = True
            #getting_closer = False
        if np.sqrt((r_rel_new**2).sum()) < np.sqrt((r_rel**2).sum()):
            getting_closer = True
        r_rel = r_rel_new
    return(getting_closer,moves_away_slightly)

moving_same_direction(star_1_vel,star_2_vel)
moving_same_direction(star_1_vel,star_3_vel)
moving_same_direction(star_2_vel,star_3_vel)




In [ ]:
print(getting_closer(star_1_vel,star_3_vel, star_1_pos,star_3_pos))
print(getting_closer(star_1_vel,star_2_vel, star_1_pos,star_2_pos))
print(getting_closer(star_2_vel,star_3_vel, star_2_pos,star_3_pos))

In [ ]:
r1 = np.array([10,0,0])
r2 = np.array([20,0,0])
v1 = np.array([100.,0,0])
v2 = np.array([50.,0,0])
getting_closer(v1,v2,r1,r2)


In [ ]:
plt.figure(figsize = (10,8))
plt.scatter(galaxy1.x[:int(n_bulge+n_disk)].value_in(units.kpc), galaxy1.y[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Big galaxy')
plt.scatter(galaxy2.x[:int(n_bulge+n_disk)].value_in(units.kpc), galaxy2.y[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Small galaxy')
plt.title("Disk + Bulge \n time = " +str(times[time]) )
plt.legend()
plt.xlabel("x [kpc]")
plt.ylabel("y [kpc]")
plt.xlim(-500,500)
#plt.axis("equal")
plt.ylim(-500,500)
#plt.savefig('./merge_plots/snapshots_xy/snap%04d.png'%time)
plt.show()
#plt.close()

plt.figure(figsize = (10,8))
plt.scatter(galaxy1.x[:int(n_bulge+n_disk)].value_in(units.kpc), galaxy1.z[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Big galaxy')
plt.scatter(galaxy2.x[:int(n_bulge+n_disk)].value_in(units.kpc), galaxy2.z[:int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Small galaxy')
plt.title("Disk + Bulge \n time = " +str(times[time]) )
plt.legend()
plt.xlabel("x [kpc]")
plt.ylabel("y [kpc]")
plt.xlim(-500,500)
#plt.axis("equal")
plt.ylim(-500,500)
#plt.savefig('./merge_plots/snapshots_yz/snap%04d.png'%time)
plt.show()
#plt.close()